In [1]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\Panel101.dta"

df, meta = st.read_dta(path)
df.head(1)

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
0,1,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,1.0,1.0,1.342788,0.0,0.0,0.0,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  70 non-null     int64  
 1   year     70 non-null     int64  
 2   y        70 non-null     float64
 3   y_bin    70 non-null     float64
 4   x1       70 non-null     float64
 5   x2       70 non-null     float64
 6   x3       70 non-null     float64
 7   opinion  70 non-null     float64
 8   op       70 non-null     float64
 9   Y        70 non-null     float64
 10  time     70 non-null     float64
 11  treated  70 non-null     float64
 12  did      70 non-null     float64
 13  _diff    70 non-null     float64
dtypes: float64(12), int64(2)
memory usage: 7.8 KB


In [3]:
meta.column_names_to_labels

{'country': 'Country',
 'year': 'Year',
 'y': 'Outcome Y',
 'y_bin': 'Binary outcome Y',
 'x1': 'Predictor x1',
 'x2': 'Predictor x2',
 'x3': 'Predictor x3',
 'opinion': 'Categorical variable',
 'op': None,
 'Y': None,
 'time': None,
 'treated': None,
 'did': None,
 '_diff': 'Diff-in-diff'}